In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [51]:

working_file_path='M2-1 Working on thesis paper.xlsx'
df = pd.read_excel(working_file_path,sheet_name='All Data')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3406 entries, 0 to 3405
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   1           3406 non-null   int64  
 1   PRICE       3406 non-null   object 
 2   BRANDNAME   3406 non-null   object 
 3   MAKEYEAR    3386 non-null   float64
 4   FUEL        3406 non-null   object 
 5   KM          2811 non-null   object 
 6   BODY        3406 non-null   object 
 7   GEARBOX     3406 non-null   object 
 8   DRIVE       3406 non-null   object 
 9   COLOR       3406 non-null   object 
 10  POSTDATE    3405 non-null   object 
 11  MILEAGE     1676 non-null   object 
 12  ENGINE(CC)  2802 non-null   object 
 13  KM_INT      2394 non-null   float64
 14  PRICE_NEW   3406 non-null   int64  
dtypes: float64(2), int64(2), object(11)
memory usage: 399.3+ KB


In [56]:
df1=df.drop('MILEAGE', axis=1)
df1=df1.drop('KM', axis=1)
df1.dropna(inplace=True)

df1.info()
# Display or use the filtered DataFrame as needed
# df2 = df.loc[:, ["MAKEYEAR", "KM_INT","PRICE_NEW","BRANDNAME","FUEL","BODY","GEARBOX","DRIVE","COLOR"]]
# df2.dropna(inplace=True)
# df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2391 entries, 0 to 3387
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   1           2391 non-null   int64  
 1   PRICE       2391 non-null   object 
 2   BRANDNAME   2391 non-null   object 
 3   MAKEYEAR    2391 non-null   float64
 4   FUEL        2391 non-null   object 
 5   BODY        2391 non-null   object 
 6   GEARBOX     2391 non-null   object 
 7   DRIVE       2391 non-null   object 
 8   COLOR       2391 non-null   object 
 9   POSTDATE    2391 non-null   object 
 10  ENGINE(CC)  2391 non-null   object 
 11  KM_INT      2391 non-null   float64
 12  PRICE_NEW   2391 non-null   int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 261.5+ KB


In [27]:
# df2 = df2.dropna(subset=['KM_INT'])

# # If you want to reset the index after removing rows
# df2 = df2.reset_index(drop=True)

pd.set_option('display.max_rows', None)


2413

In [141]:
#Removing all empty columns except KM_INT(distance travelled)

withNullKMDataFrame = newDf.dropna(subset=['MAKEYEAR', 'BODY','DRIVE','FUEL','BRANDNAME','GEARBOX','PRICE_NEW'])
len(withNullKMDataFrame)

3496

In [142]:
#Drop specific Rows  with specific columns That  are null or empty

withoutNullKMDataFrame = newDf.dropna(subset=['MAKEYEAR', 'BODY','DRIVE','FUEL','BRANDNAME','GEARBOX','KM_INT','PRICE_NEW'])
len(withoutNullKMDataFrame)

2865

In [143]:
withoutNullKMDataFrame['COLOR'].value_counts()

COLOR
White               544
Silver              395
Grey                152
Other               146
silver              140
                   ... 
ALICE BLUE            1
Urban Titanium        1
bule                  1
light Blue            1
Moon dust Silver      1
Name: count, Length: 425, dtype: int64

In [ ]:
#plot the data in scatter plot
import matplotlib.pyplot as plt


plt.scatter(df2["MAKEYEAR"], df2["KM_INT"])
plt.show()

In [ ]:
#Running Linear regression for Make year and km

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

X = df2['MAKEYEAR']
y = df2["KM_INT"]
# print(X,y);
sns.lmplot(x='MAKEYEAR', y='KM_INT', data=df2, ci=None, line_kws={'color': 'red'})
plt.xlabel('Year')
plt.ylabel('Distance Travelled')
plt.title('Linear Regression: Year vs Distance Travelled')
plt.show()

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt

years_2d = np.array(df2['MAKEYEAR']).reshape(-1, 1)
X = years_2d  # Features
y = df2['KM_INT']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# By Earlier model using training set
# # Train the model
# model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = model.predict(X_test)
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
# print('Root Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred, squared=False))


# By Sagar Chapagain using initial set
model.fit(X, y)
y_pred = model.predict(X)

r2=r2_score(y, y_pred);

print("Number of samples in X:", y_pred.shape[0])
print("Number of samples in y:", y.shape[0])
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
# print('Root Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred, squared=False))
print("R2 score is : ",r2)

#calculate r squared value
# Evaluate the model

# Plot the regression line
plt.scatter(X_train, y_test, color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)
plt.xlabel('Year')
plt.ylabel('Distance Travelled')
plt.title('Linear Regression: Year vs Distance Travelled')
plt.show()

In [ ]:
df2["MAKEYEAR"].value_counts()


In [ ]:
df2["KM_INT"].describe()

In [ ]:
zerokm_filter = (df2["KM_INT"] == 0)

df2[zerokm_filter]

https://www.youtube.com/watch?v=Wqmtf9SA_kk&ab_channel=NeuralNine
https://www.youtube.com/watch?v=xu0prYu5e-c&ab_channel=AliNasserEddine